In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import numpy as np
import argparse
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_svmlight_file
import sys
sys.path.append('../')

from cs_opl import op_learning
from experiment_learning import *

In [ ]:
data_name = 'satimage'
num_trials = 1
sample_size = 300

if data_name == 'satimage':
    data_name = 'satimage.scale'
elif data_name == 'vehicle':
    data_name = 'vehicle.scale'

alphas = [0.7, 0.4, 0.0]

tau_list = np.zeros(num_trials)
res_ipw3_list = np.zeros((num_trials, len(alphas)))
res_dm_list = np.zeros((num_trials, len(alphas)))
res_dml2_list = np.zeros((num_trials, len(alphas)))

res_ipw3_sn_list = np.zeros((num_trials, len(alphas)))
res_dml2_sn_list = np.zeros((num_trials, len(alphas)))

for trial in range(num_trials):
    X, Y, Y_matrix, train_test_split, classes, N, N_train, N_test = data_generation(data_name, sample_size)

    X_train, X_test = X[train_test_split], X[~train_test_split]

    Y_matrix_train, Y_matrix_test = Y_matrix[train_test_split], Y_matrix[~train_test_split]

    for idx_alpha in  range(len(alphas)):    
        alpha = alphas[idx_alpha]

        pi_behavior, pi_evaluation  = behavior_and_evaluation_policy(X, Y, train_test_split, classes, alpha=alpha)

        pi_behavior_train = pi_behavior[train_test_split]
        pi_evaluation_train, pi_evaluation_test = pi_evaluation[train_test_split], pi_evaluation[~train_test_split]

        tau = true_value(Y_matrix_test, pi_evaluation_test, N_test)
        tau_list[trial] = tau

        perm = np.random.permutation(N_train)
        X_seq_train, Y_matrix_seq_train, pi_behavior_seq_train, pi_evaluation_seq_train = X_train[perm], Y_matrix_train[perm], pi_behavior_train[perm], pi_evaluation_train[perm]

        Y_historical_matrix = np.zeros(shape=(N_train, len(classes)))
        A_historical_matrix = np.zeros(shape=(N_train, len(classes)))

        for i in range(N_train):
            a = np.random.choice(classes, p=pi_behavior_seq_train[i])
            Y_historical_matrix[i, a] = Y_matrix_seq_train[i, a]
            A_historical_matrix[i, a] = 1
            
        estimators = op_learning(X_seq_train, A_historical_matrix, Y_historical_matrix, X_test, classes)
        
        estimators.ipw_est_parameters()
        epol_ipw = estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=False)
        #epol_ipw_sn = estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=True)

        estimators.dm_est_parameters()
        epol_dm = estimators.dm_fit(folds=5, algorithm='Ridge')
        
        estimators.dml_est_parameters(folds=5, algorithm='Ridge')
        epol_dml = estimators.dml_fit(folds=5, algorithm='Ridge', self_norm=False)
        epol_dml_sn = estimators.dml_fit(folds=5, algorithm='Ridge', self_norm=True)

        res_ipw3 =  true_value(Y_matrix_test, epol_ipw, N_test)
        res_ipw3_sn =  true_value(Y_matrix_test, epol_ipw_sn, N_test)
        res_dm =  true_value(Y_matrix_test, epol_dm, N_test)
        res_dml2 =  true_value(Y_matrix_test, epol_dml, N_test)
        res_dml2_sn =  true_value(Y_matrix_test, epol_dml_sn, N_test)

        print('trial', trial)
        print('True:', tau)
        print('IPW3:', res_ipw3)
        print('IPW3_SN:', res_ipw3_sn)
        print('DM:', res_dm)
        print('DML2:', res_dml2)
        print('DML2_SN:', res_dml2_sn)

        res_ipw3_list[trial, idx_alpha] = res_ipw3
        res_ipw3_sn_list[trial, idx_alpha] = res_ipw3_sn
        res_dm_list[trial, idx_alpha] = res_dm
        res_dml2_list[trial, idx_alpha] = res_dml2
        res_dml2_sn_list[trial, idx_alpha] = res_dml2_sn

        np.savetxt("exp_results/true_value_%s.csv"%data_name, tau_list, delimiter=",")
        np.savetxt("exp_results/res_opl_ipw3_%s.csv"%data_name, res_ipw3_list, delimiter=",")
        np.savetxt("exp_results/res_opl_ipw3_sn_%s.csv"%data_name, res_ipw3_sn_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dm_%s.csv"%data_name, res_dm_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dml2_%s.csv"%data_name, res_dml2_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dml2_sn_%s.csv"%data_name, res_dml２_sn_list, delimiter=",")

In [ ]:
epol_dml = estimators.dml_fit(folds=5, algorithm='Ridge', self_norm=False)

score0 0.6281391056431078
score 0.611430644988967
score0 0.42378399295433766
score 0.6225404625964648
score0 0.37013128600914963
score 0.5321241309923755
score0 0.1444720140860965
score 0.18152727628563511
score0 0.1199047602761341
score 0.1448962006648356
score0 0.6307189979415473
score 0.4666335542255565
score0 0.46211439254469266
score 0.47438096205368513
score0 0.40743386605520443
score 0.41370491617286415
score0 0.15641548884902803
score 0.15441627150564424
score0 0.1256774783461448
score 0.12277383666902063
score0 0.6598423620621678
score 0.37159114994547726
score0 0.48737779052865676
score 0.3761616118631341
score0 0.43208402988614975
score 0.3331174965241599
score0 0.16447692960961285
score 0.1382561263029049
score0 0.12855572508175017
score 0.11205811736380601
score0 0.4967873733363951
score 0.3765508167144877
score0 0.4888873575311859
score 0.3702025919112345
score0 0.4333972118401222
score 0.32794964015250205
score0 0.16529144059203515
score 0.1363244045817416
score0 0.12860